In [2]:
import torch

print("torch version is : ", torch.__version__)

torch version is :  2.7.1+cu118


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

transformation = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(
    root='data',
    train=True,
    download=True,
    transform=transformation,
)

test_dataset = datasets.CIFAR10(
    root='data',
    train=False,
    download=True,
    transform=transformation,

)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True,    num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False,     num_workers=2)

print(f"Training data shape: {train_dataset.data.shape}")       # (50000, 32, 32, 3)
print(f"Training labels shape: {len(train_dataset.targets)}")   # 50000
print(f"Test data shape: {test_dataset.data.shape}")            # (10000, 32, 32, 3)
print(f"Test labels shape: {len(test_dataset.targets)}")        # 10000

print(f"Type of train_dataset.targets: {type(train_dataset.targets)}")
print(f"Type of train_dataset.data: {type(train_dataset.data)}")
print(f"Length of targets (list): {len(train_dataset.targets)}")
print(f"Shape of data (numpy): {train_dataset.data.shape}")
print(f"First 5 targets: {train_dataset.targets[:5]}")

Training data shape: (50000, 32, 32, 3)
Training labels shape: 50000
Test data shape: (10000, 32, 32, 3)
Test labels shape: 10000
Type of train_dataset.targets: <class 'list'>
Type of train_dataset.data: <class 'numpy.ndarray'>
Length of targets (list): 50000
Shape of data (numpy): (50000, 32, 32, 3)
First 5 targets: [6, 9, 9, 4, 1]


In [10]:
import torch.nn as nn


class deep_CNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size =3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size =3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25)

        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128,10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

from torchsummary import summary
model = deep_CNN().to(device)

summary(model, input_size=(3, 32, 32))
print(model)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
           Dropout-5           [-1, 32, 16, 16]               0
            Conv2d-6           [-1, 64, 16, 16]          18,496
       BatchNorm2d-7           [-1, 64, 16, 16]             128
              ReLU-8           [-1, 64, 16, 16]               0
         MaxPool2d-9             [-1, 64, 8, 8]               0
          Dropout-10             [-1, 64, 8, 8]               0
          Flatten-11                 [-1, 4096]               0
           Linear-12                  [-1, 128]         524,416
      BatchNorm1d-13                  [-1, 128]             256
             ReLU-14                  [

In [16]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [18]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images,labels in train_loader:
        images,labels = images.to(device),labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}] Loss: {running_loss / len(train_loader):.4f}")

Epoch [1/10] Loss: 0.6495
Epoch [2/10] Loss: 0.6266
Epoch [3/10] Loss: 0.6088
Epoch [4/10] Loss: 0.5870
Epoch [5/10] Loss: 0.5645
Epoch [6/10] Loss: 0.5487
Epoch [7/10] Loss: 0.5343
Epoch [8/10] Loss: 0.5201
Epoch [9/10] Loss: 0.5052
Epoch [10/10] Loss: 0.4937


In [19]:
model.eval()

all_prediction = []
all_labels = []

with torch.no_grad():

    for images,labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs,dim=1)

            all_prediction.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

In [20]:
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(all_labels, all_prediction))

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.68      0.76      1000
           1       0.91      0.83      0.87      1000
           2       0.77      0.55      0.64      1000
           3       0.60      0.56      0.58      1000
           4       0.60      0.86      0.71      1000
           5       0.67      0.69      0.68      1000
           6       0.80      0.84      0.82      1000
           7       0.78      0.84      0.81      1000
           8       0.81      0.90      0.85      1000
           9       0.85      0.85      0.85      1000

    accuracy                           0.76     10000
   macro avg       0.77      0.76      0.76     10000
weighted avg       0.77      0.76      0.76     10000

